# c05-plates

*Purpose*: 


## Informed Consent

As a reminder, this course is part of a study of engineers' behavior. While not all parts of the course include data collection, we will analyze your responses to this homework as part of the research.

We will analyze your answers to this homework, and may quote this work as part of published research.

You can ask to have your responses excluded from the study after the interview by sending us an email. Before starting this assignment, do you consent to sharing your work with the study?


I agree to share my responses with the study

- (Please type your name here)


In [ ]:
import grama as gr
import pandas as pd
DF = gr.Intention()
%matplotlib inline

# For assertion
from pandas.api.types import is_numeric_dtype


## Background

(This continues the stang challenge.)


# Assess Statistical Control


In [ ]:
from grama.data import df_stang


### __qX__ Assess statistical control across thicknesses


In [ ]:
(
    df_stang
    >> gr.pt_xbs(group="thick", var="E")
)

*Observations*

- Is the variability of `E` under statistical control across plate thicknesses? How do you know?
  - (Your response here)
- Is the mean of `E` under statistical control across plate thicknesses? How do you know?
  - (Your response here)


## Follow-up experiment

*Note*: The following data were simulated; they do not correspond to physical experiments.


In [ ]:
df_data = pd.read_csv("./data/c05-data.csv")
df_data

| Symbol | Variable | Meaning |
|---|---|---|
| `E` | Elasticity (ksi) | Mechanical property |
| `mu` | Poisson's ratio (-) | Mechanical property |
| `t` | Thickness (in) | Geometric property |
| `id_machine` | Unique machine identifier | Manufacturing variable |
| `id_specimen` | Unique specimen identifier | Manufacturing variable |
| `id_measurement` | Unique measurement (operator) identifier | Manufacturing variable |


### __qX__ Explore the experimental design

Answer the following questions to better understand the experimental design. Note that the same questions are posed within each cell and under *observations* below.

*Hint*: The verbs `tf_count()` and `tf_distinct()` will be very useful for answering some of these questions!


In [ ]:
## Q: What thicknesses were tested?


In [ ]:
## Q: How many unique specimens were manufactured?


In [ ]:
## Q: How many specimens were made on each machine?


In [ ]:
## Q: How many times did each operator measure each specimen?


*Observations*

- What thicknesses were tested?
  - (Your response here)
- How many unique specimens were manufactured?
  - (Your response here)
- How many specimens were made on each machine?
  - (Your response here)
- How many times did each operator measure each specimen?
  - (Your response here)


### __qX__ Compare across thicknesses

Compare the elasticity across plate thicknesses; does elasticity seem to be consistent across thickness?


In [ ]:
(
    df_data
    >> gr.ggplot(gr.aes("t", "E"))
    + gr.geom_boxplot(gr.aes(group="t"), notch=True)
)

*Observations*

- Is thickness consistent across plate thickness?
  - (Your response here)
- Will it be reasonable to group together plates of different thicknesses when assessing statistical control? Why or why not?
  - (Your response here)


### __qX__ Assess statistical control of Poisson's ratio

Consider only the `t == 0.250` plates. Assess the state of statistical control of Poisson's ratio. Answer the questions under *observations* below.


In [ ]:
## TODO: Assess the state of statistical control


*Observations*

- Is `mu` under statistical control? Why or why not?
  - (Your response here)


### __qX__ Assess statistical control of elasticity

Consider only the `t == 0.250` plates. Assess the state of statistical control of the elasticity. Answer the questions under *observations* below.


In [ ]:
## TODO: Assess the state of statistical control


*Observations*

- Why is it important that we limit this analysis to `t == 0.25` plates?
  - (Your response here)
- Is `E` under statistical control? Why or why not?
  - (Your response here)
- Based on the group variable(s) you chose, what follow-up investigations should be done?
  - (Your response here)


### __qX__ Assess statistical control of elasticity (Pt. 2)





In [ ]:
(
    df_data
    >> gr.tf_filter(
        DF.t == 0.250,
        DF.id_machine != "C",
        DF.id_measurement != "h",
    )

)

In [ ]:
(
    df_data
    >> gr.tf_filter(
        DF.id_machine != "C",
        DF.id_measurement != "h",
    )
    
    >> gr.pt_xbs(group="id_measurement", var="E")
)

*Observations*

- Is `E` under statistical control? Why or why not?
  - (Your response here)
- Based on the group variable(s) you chose, what follow-up investigations should be done?
  - (Your response here)


# Consider Sources of Variability

(TODO)

For the rest of the exercise, we will consider the following subset of the data.


In [ ]:
## NOTE: No need to edit
df_sub = (
    df_data
    >> gr.tf_filter(
        DF.t == 0.250,
        DF.id_machine != "C",
        DF.id_measurement != "h",
    )
)

### __qX__ Estimate the real variability

Identify the column in `df_sub` that groups together multiple measurements of the same quantity.

The code below applies the *mean heuristic* ([e-stat02-source](https://zdelrosario.github.io/evc-course/exercises_solution/d08-e-stat02-source-solution.html#heuristics)) to produce a more stable measurement, then compute the variance across these more stable measurements.


In [ ]:
## TASK: Apply the Mean Heuristic to group by the appropriate variable
df_var_mfg = (
    df_sub
    ## TODO: Group by the appropriate variable

    >> gr.tf_summarize(E=gr.mean(DF.E))
    >> gr.tf_ungroup()
    >> gr.tf_summarize(E_var_mfg=gr.var(DF.E))
)

## NOTE: No need to edit; use this to check your work
assert \
    abs(df_var_mfg.E_var_mfg[0] - 109670.635716) < 1e-6, \
    "Incorrect variance; make sure you grouped by the correct variable"

df_var_mfg

### __qX__ Estimate the measurement variability


In [ ]:
df_var_meas = (
    df_sub

)

## NOTE: No need to edit; use this to check your work
assert \
    abs(df_var_meas.iloc[0, 0] - 168382.91387) < 1e-6, \
    "Incorrect variance; make sure you grouped by the correct variable"

df_var_meas

In [ ]:
df_real = (
    df_sub
    >> gr.tf_group_by(DF.id_specimen)
    >> gr.tf_summarize(
        mu=gr.mean(DF.mu),
        E=gr.mean(DF.E),
    )
    >> gr.tf_ungroup()
)

# Model Variability


### __qX__ Assess dependency of `E` and `mu`


In [ ]:
(
    df_sub
    >> gr.ggplot(gr.aes("E", "mu"))
    + gr.geom_point()
)

In [ ]:
md_plate = (
    gr.Model("Plate critical buckling stress")
    >> gr.cp_vec_function(
        fun=lambda df: gr.df_make(
            k_cr=(df.m * df.b / df.a + df.a / df.m / df.b)**2
        ),
        var=["a", "b", "m"],
        out=["k_cr"],
        name="Shape factor",
    )
    >> gr.cp_vec_function(
        fun=lambda df: gr.df_make(
            sigma_cr=df.k_cr * (3.14**3/12) * df.E*1e3 / (1 - df.mu**2)
                    *(df.t / df.b)**2
        ),
        var=["k_cr", "E", "mu", "t", "b"],
        out=["sigma_cr"],
        name="Buckling stress",
    )
    >> gr.cp_vec_function(
        fun=lambda df: gr.df_make(
            g_buckle=df.sigma_cr - 2e5 / df.b / df.t,
        ),
        var=["sigma_cr", "b", "t"],
        out=["g_buckle"],
        name="Limit state: Buckling",
    )
)

md_plate

### __qX__ Fit with all observations


In [ ]:
md_total = (
    md_plate

)
md_total

### __qX__ Fit with stabilized observations


In [ ]:
md_real = (
    md_plate

)
md_real

# Design Under Uncertainty


### __qX__ Assess a baseline design


In [ ]:
df_baseline_total = (
    md_total
    >> gr.ev_sample(
        n=1e3,
        df_det=gr.df_make(t=0.25, a=12.0, b=9.0, m=1),
    )
    >> gr.tf_summarize(
        pof_lo=gr.pr_lo(DF.g_buckle <= 0),
        pof=gr.pr(DF.g_buckle <= 0),
        pof_up=gr.pr_up(DF.g_buckle <= 0),
    )
)

In [ ]:
df_baseline_real = (
    md_real
    >> gr.ev_sample(
        n=1e3,
        df_det=gr.df_make(t=0.25, a=12.0, b=9.0, m=1),
    )
    >> gr.tf_summarize(
        pof_lo=gr.pr_lo(DF.g_buckle <= 0),
        pof=gr.pr(DF.g_buckle <= 0),
        pof_up=gr.pr_up(DF.g_buckle <= 0),
    )
)

In [ ]:
(
    df_baseline_total
)